In [0]:

!git clone --recursive https://github.com/krosac/Pruning_PJ3 Pruning
!git clone https://github.com/lmEshoo/pruning 
#%cp -rf drive/*/Pruning .
%ls 
%cd Pruning/cifar10/
%cp -rf ../../pruning/ckpt .
!sh get_datasets.sh
!pwd


Cloning into 'Pruning'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 160 (delta 48), reused 87 (delta 19), pack-reused 0
Receiving objects: 100% (160/160), 3.15 MiB | 7.80 MiB/s, done.
Resolving deltas: 100% (48/48), done.
drive/  Pruning/  sample_data/
/content/Pruning/cifar10
--2020-03-22 10:41:36--  http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz’

cifar-10-python.tar 100%[===================>] 162.60M  46.1MB/s    in 3.7s    

2020-03-22 10:41:40 (44.1 MB/s) - ‘cifar-10-python.tar.gz’ saved [170498071/170498071]

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.

In [0]:
#@title Default title text

""" 
Code for loading CIFAR10 and training was originally written 
for CS 231n at Stanford University (cs231n.stanford.edu). 
It has been modified for pruning.
For the original version, please visit cs231n.stanford.edu.  
"""
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
import os
import shutil

#from data import get_CIFAR10_data
#from model import model
#from prune_utils import get_prune_op
from cs231n.data_utils import load_CIFAR10

###################################
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=10000):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image

    return X_train, y_train, X_val, y_val, X_test, y_test


########################################
##  MODEL   Mobilenet
########################################

weight_decay=1e-4

def relu(x, name='relu6'):
    return tf.nn.relu6(x, name)


def batch_norm(x, momentum=0.9, epsilon=1e-5, train=True, name='bn'):
    return tf.layers.batch_normalization(x,
                      momentum=momentum,
                      epsilon=epsilon,
                      scale=True,
                      training=train,
                      name=name)


def conv2d(input_, output_dim, k_h, k_w, d_h, d_w, stddev=0.02, name='conv2d', bias=False):
    print("convd")
    with tf.variable_scope(name):
        print("convd_mid")
        print(input_)
        print( output_dim, k_h, k_w, d_h, d_w)
        w = tf.get_variable('w', [k_h, k_w, input_.get_shape()[-1], output_dim],
              regularizer=tf.contrib.layers.l2_regularizer(weight_decay),
              initializer=tf.truncated_normal_initializer(stddev=stddev))
        conv = tf.nn.conv2d(input_, w, strides=[1, d_h, d_w, 1], padding='SAME')
        if bias:
            biases = tf.get_variable('bias', [output_dim], initializer=tf.constant_initializer(0.0))
            conv = tf.nn.bias_add(conv, biases)
        print("convd_end")
        return conv


def conv2d_block(input, out_dim, k, s, is_train, name):
    with tf.name_scope(name), tf.variable_scope(name):
        net = conv2d(input, out_dim, k, k, s, s, name='conv2d')
        net = batch_norm(net, train=is_train, name='bn')
        net = relu(net)
        return net


def conv_1x1(input, output_dim, name, bias=False):
    print("conv_1x1")
    with tf.name_scope(name):
        return conv2d(input, output_dim, 1,1,1,1, stddev=0.02, name=name, bias=bias)

def pwise_block(input, output_dim, is_train, name, bias=False):
    with tf.name_scope(name), tf.variable_scope(name):
        out=conv_1x1(input, output_dim, bias=bias, name='pwb')
        out=batch_norm(out, train=is_train, name='bn')
        out=relu(out)
        return out


def dwise_conv(input, k_h=3, k_w=3, channel_multiplier= 1, strides=[1,1,1,1],
               padding='SAME', stddev=0.02, name='dwise_conv', bias=False):
    with tf.variable_scope(name):
        in_channel=input.get_shape().as_list()[-1]
        w = tf.get_variable('w', [k_h, k_w, in_channel, channel_multiplier],
                        regularizer=tf.contrib.layers.l2_regularizer(weight_decay),
                        initializer=tf.truncated_normal_initializer(stddev=stddev))
        conv = tf.nn.depthwise_conv2d(input, w, strides, padding, rate=None,name=None,data_format=None)
        if bias:
            biases = tf.get_variable('bias', [in_channel*channel_multiplier], initializer=tf.constant_initializer(0.0))
            conv = tf.nn.bias_add(conv, biases)

        return conv


def res_block(input, expansion_ratio, output_dim, stride, is_train, name, bias=False, shortcut=True):
    with tf.name_scope(name), tf.variable_scope(name):
        # pw
        bottleneck_dim=round(expansion_ratio*input.get_shape().as_list()[-1])
        net = conv_1x1(input, bottleneck_dim, name='pw', bias=bias)
        net = batch_norm(net, train=is_train, name='pw_bn')
        net = relu(net)
        # dw
        net = dwise_conv(net, strides=[1, stride, stride, 1], name='dw', bias=bias)
        net = batch_norm(net, train=is_train, name='dw_bn')
        net = relu(net)
        # pw & linear
        net = conv_1x1(net, output_dim, name='pw_linear', bias=bias)
        net = batch_norm(net, train=is_train, name='pw_linear_bn')

        # element wise add, only for stride==1
        if shortcut and stride == 1:
            in_dim=int(input.get_shape().as_list()[-1])
            if in_dim != output_dim:
                ins=conv_1x1(input, output_dim, name='ex_dim')
                net=ins+net
            else:
                net=input+net

        return net


def separable_conv(input, k_size, output_dim, stride, pad='SAME', channel_multiplier=1, name='sep_conv', bias=False):
    with tf.name_scope(name), tf.variable_scope(name):
        in_channel = input.get_shape().as_list()[-1]
        dwise_filter = tf.get_variable('dw', [k_size, k_size, in_channel, channel_multiplier],
                  regularizer=tf.contrib.layers.l2_regularizer(weight_decay),
                  initializer=tf.truncated_normal_initializer(stddev=0.02))

        pwise_filter = tf.get_variable('pw', [1, 1, in_channel*channel_multiplier, output_dim],
                  regularizer=tf.contrib.layers.l2_regularizer(weight_decay),
                  initializer=tf.truncated_normal_initializer(stddev=0.02))
        strides = [1,stride, stride,1]

        conv=tf.nn.separable_conv2d(input,dwise_filter,pwise_filter,strides,padding=pad, name=name)
        if bias:
            biases = tf.get_variable('bias', [output_dim],initializer=tf.constant_initializer(0.0))
            conv = tf.nn.bias_add(conv, biases)
        return conv


def global_avg(x):
    with tf.name_scope('global_avg'):
        net=tf.layers.average_pooling2d(x, x.get_shape()[1:-1], 1)
        return net


def flatten(x):
    print("flatten")
    #flattened=tf.reshape(input,[x.get_shape().as_list()[0], -1])  # or, tf.layers.flatten(x)
    return tf.contrib.layers.flatten(x)



def pad2d(inputs, pad=(0, 0), mode='CONSTANT'):
    paddings = [[0, 0], [pad[0], pad[0]], [pad[1], pad[1]], [0, 0]]
    net = tf.pad(inputs, paddings, mode=mode)
    return net



def model_mobilenetv2(X, y, is_train=True, reuse=tf.AUTO_REUSE):
    exp = 6  # expansion ratio
    with tf.variable_scope('mobilenetv2'):
        net = conv2d_block(X, 32, 3, 2, is_train, name='conv1_1')  # size/2
        print("hello1")
        net = res_block(net, 1, 16, 1, is_train, name='res2_1')

        net = res_block(net, exp, 24, 2, is_train, name='res3_1')  # size/4
        net = res_block(net, exp, 24, 1, is_train, name='res3_2')
        print("hello2")
        net = res_block(net, exp, 32, 2, is_train, name='res4_1')  # size/8
        net = res_block(net, exp, 32, 1, is_train, name='res4_2')
        net = res_block(net, exp, 32, 1, is_train, name='res4_3')

        net = res_block(net, exp, 64, 2, is_train, name='res5_1')
        net = res_block(net, exp, 64, 1, is_train, name='res5_2')
        net = res_block(net, exp, 64, 1, is_train, name='res5_3')
        net = res_block(net, exp, 64, 1, is_train, name='res5_4')
        print("hello3")
        net = res_block(net, exp, 96, 1, is_train, name='res6_1')  # size/16
        net = res_block(net, exp, 96, 1, is_train, name='res6_2')
        net = res_block(net, exp, 96, 1, is_train, name='res6_3')

        net = res_block(net, exp, 160, 2, is_train, name='res7_1')  # size/32
        net = res_block(net, exp, 160, 1, is_train, name='res7_2')
        net = res_block(net, exp, 160, 1, is_train, name='res7_3')

        net = res_block(net, exp, 320, 1, is_train, name='res8_1', shortcut=False)
        print("hello4")
        net = pwise_block(net, 1280, is_train, name='conv9_1')
        print("hello4_2")
        net = global_avg(net)
        print("hello4_3")
        logits = flatten(conv_1x1(net, y, name='logits'))
        print("hello4_4")
        #pred = tf.nn.softmax(logits, name='prob')
        y_out = tf.nn.softmax(logits, name='prob')
        print("hello5")
        #return logits, pred
        #return y_out
        return logits


#def model(X,y):
#    # define our weights (e.g. init_two_layer_convnet)    
#    # setup variables
#    Wconv1 = tf.get_variable("Wconv1_4", shape=[7, 7, 3, 32])
#    bconv1 = tf.get_variable("bconv1_4", shape=[32])
#    W1 = tf.get_variable("W1_4", shape=[5408, 10])
#    b1 = tf.get_variable("b1_4", shape=[10])

#    # define our graph (e.g. two_layer_convnet)
#    a1 = tf.nn.conv2d(X, Wconv1, strides=[1,2,2,1], padding='VALID') + bconv1
#    h1 = tf.nn.relu(a1)
#    h1_flat = tf.reshape(h1,[-1,5408])
#    y_out = tf.matmul(h1_flat,W1) + b1
#    return y_out


########################################

def get_prune_op(sess, train_gradient, percentage):
    #all_variables = sess.graph.get_collection('variables')
    all_variables = [v for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES) if v.name.endswith('w:0')]
    print(all_variables)
    print(train_gradient)
    #train_gradient_f = [v for v in train_gradient if v[1].name.endswith('w:0')]
    prune_gradient =  train_gradient
    with sess.as_default():
        #print(all_variables)
        for i,var in enumerate(prune_gradient):
            if var[1].name.endswith('w:0'):
                print(var)
                var_np = var[1].eval()
                grad = var[0]
                #print(var_np)
                #print(var_np.shape)
                percent = percentage * 100
                thresh = np.percentile(np.absolute(var_np), percent)
                print(thresh , percent)
                mask = np.zeros(var_np.shape, dtype=np.float32)
                #mask = [1 if a_ > thresh else a_ for a_ in mask]
                super_threshold_indices = np.absolute(var_np) > thresh
                #print(super_threshold_indices)
                mask[super_threshold_indices] = 1
                #print(mask)
                new_var_np = mask * var_np
                prune_op = var[1].assign(new_var_np)
                sess.run(prune_op)
                mask_tensor = tf.convert_to_tensor(mask)
                prune_gradient[i] = [tf.multiply(grad, mask_tensor), var[1]]
          

    #a = [0 if a_ > thresh else a_ for a_ in a]  
    #init = tf.global_variables_initializer()
    #sess.run(init)
    #v_1 = mobilenetv2/conv1_1/conv2d/w.eval()
    #print(v_1)
    # ================================================================ #
    # YOUR CODE HERE:
    #   1.find prunable variables i.e. kernel weight/bias
    #   2.prune parameters based on your threshold, calculated based on input argument percentage!
    #   3.get pruned gradient update operator accordingly, save to prune_gradient
    #   example pseudo code for step 2-3:
    #       for var in prunable_variables:
    #           var_np = var.eval()
    #           # construct pruning mask
    #           new_var_np = mask * var_np
    #           prune_op = var.assign(new_var_np)
    #           # apply parameter pruning by sess.run(prune_op)
    #           prune_gradient = train_gradient * mask for each parameter
    #               Example:
    #                   mask = np.zeros(96, dtype=np.float32)
    #                   mask_tensor = tf.convert_to_tensor(mask)
    #                   prune_gradient = [[tf.multiply(v[0], mask_tensor), v[1]] for v in train_gradient if 'fcn/conv1-1/bias' in v[1].name]
    #        return prune_gradient   
    # ================================================================ #
    
    # modify here, no change on gradient for now
    #prune_gradient = train_gradient
    
    # ================================================================ #
    # END YOUR CODE HERE
    # ================================================================ #
    return prune_gradient


##########################################




In [0]:
""" 
Code for loading CIFAR10 and training was originally written 
for CS 231n at Stanford University (cs231n.stanford.edu). 
It has been modified for pruning.
For the original version, please visit cs231n.stanford.edu.  
"""
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
import os
import shutil


class Model:
    def __init__(self):
        # gpu option
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True 
        self.sess = tf.Session(config=config)
    
    def load_cifar10(self):        
        # Invoke the above function to get our data.
        self.X_train, self.y_train, self.X_val, self.y_val, self.X_test, self.y_test = get_CIFAR10_data()
        print('Train data shape: ', self.X_train.shape)
        print('Train labels shape: ', self.y_train.shape)
        print('Validation data shape: ', self.X_val.shape)
        print('Validation labels shape: ', self.y_val.shape)
        print('Test data shape: ', self.X_test.shape)
        print('Test labels shape: ', self.y_test.shape)
        
    def construct_model(self, ckpt_dir=None):
        # Define inputs
        self.X = tf.placeholder(tf.float32, [None, 32, 32, 3])
        self.y = tf.placeholder(tf.int64, [None])
        self.is_training = tf.placeholder(tf.bool)
        self.global_step = tf.train.get_or_create_global_step()

        # ================================================================ #
        # YOUR CODE HERE:
        #   define our model
        #   save output of the model to self.y_out 
        # ================================================================ #
        
        #self.y_out = model(self.X,self.y)
        ########## Instead of self.y , number of output classes to be written which is 10
        self.y_out = model_mobilenetv2(self.X,10)
        #self.y_out = model_conv_net(self.X, weights, biases)
        print(self.y_out)
        # ================================================================ #
        # END YOUR CODE HERE
        # ================================================================ #

        # Define our loss
        total_loss = tf.losses.softmax_cross_entropy(tf.one_hot(self.y,10),logits=self.y_out)
        self.mean_loss = tf.reduce_mean(total_loss)

        # Define our optimizer
        self.optimizer = tf.train.AdamOptimizer(5e-4) # select optimizer and set learning rate
            
        # split train_step = optimizer.minimize(self.mean_loss)
        train_gradient = self.optimizer.compute_gradients(self.mean_loss)
        
        # initialize or load model parameters
        self.saver = tf.train.Saver(max_to_keep=10)
        if ckpt_dir is not None:
            self.saver.restore(self.sess, tf.train.latest_checkpoint(ckpt_dir))
            print('Pre-trained model restored from %s' % (ckpt_dir))
        else:
            self.sess.run(tf.global_variables_initializer())
            print('Initialize variables')
        # ================================================================ #
        # YOUR CODE HERE:
        #   implement in prune_utils.py
        #   1.prune parameters based on your threshold 
        #     (make sure pruning is effectively applied in step 1)
        #   2.get pruned gradient update operator accordingly, save to prune_gradient
        # ================================================================ #
        
        prune_gradient = get_prune_op(self.sess, train_gradient, percentage=0.8)
        print(prune_gradient)
        # ================================================================ #
        # END YOUR CODE HERE
        # ================================================================ #
        # save pruned parameters
        tmp_dir = '__tmp__'
        if not os.path.exists(tmp_dir):
            os.makedirs(tmp_dir)
        self.saver.save(self.sess, os.path.join(tmp_dir, 'pruned_model.ckpt'))       
        # define gradients and initialize optimizer parameters
        self.train_op = self.optimizer.apply_gradients(prune_gradient, global_step=self.global_step)
        self.sess.run(tf.global_variables_initializer())
        # reload pruned parameters
        self.saver.restore(self.sess, tf.train.latest_checkpoint(tmp_dir))
        if os.path.exists(tmp_dir):
            shutil.rmtree(tmp_dir)        
        
        
    def run_model(self, sess, predict, loss_val, Xd, yd,
              epochs=1, batch_size=64, print_every=100,
              save_every=1, train_op=None, plot_losses=False):
        # have tensorflow compute accuracy
        correct_prediction = tf.equal(tf.argmax(predict,1), self.y)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        # shuffle indicies
        train_indicies = np.arange(Xd.shape[0])
        np.random.shuffle(train_indicies)

        training_now = train_op is not None
        
        # setting up variables we want to compute (and optimizing)
        # if we have a training function, add that to things we compute
        variables = [loss_val,correct_prediction,accuracy]
        if training_now:
            variables[-1] = train_op

        # counter 
        iter_cnt = 0
        for e in range(epochs):
            # keep track of losses and accuracy
            correct = 0
            losses = []
            # make sure we iterate over the dataset once
            for i in range(int(math.ceil(Xd.shape[0]/batch_size))): # TODO: ceil?? Is this right?!
                # generate indicies for the batch
                start_idx = (i*batch_size)%Xd.shape[0]
                idx = train_indicies[start_idx:start_idx+batch_size]
                
                # create a feed dictionary for this batch
                feed_dict = {self.X: Xd[idx,:],
                             self.y: yd[idx],
                             self.is_training: training_now }
                # get batch size
                actual_batch_size = yd[idx].shape[0]
                
                # have tensorflow compute loss and correct predictions
                # and (if given) perform a training step
                loss, corr, _ = sess.run(variables,feed_dict=feed_dict)
                
                # aggregate performance stats
                losses.append(loss*actual_batch_size)
                correct += np.sum(corr)
                
                # print every now and then
                if training_now and (iter_cnt % print_every) == 0:
                    print("Iteration {0}: with minibatch training loss = {1:.3g} and accuracy of {2:.2g}"\
                          .format(iter_cnt,loss,np.sum(corr)/actual_batch_size))
                iter_cnt += 1
            total_correct = correct/Xd.shape[0]
            total_loss = np.sum(losses)/Xd.shape[0]
            print("Epoch {2}, Overall loss = {0:.3g} and accuracy of {1:.3g}"\
                  .format(total_loss,total_correct,e+1))
            if plot_losses:
                plt.plot(losses)
                plt.grid(True)
                plt.title('Epoch {} Loss'.format(e+1))
                plt.xlabel('minibatch number')
                plt.ylabel('minibatch loss')
                plt.show()
            # Save the model checkpoint periodically.
            if e % save_every == 0:
                checkpoint_path = os.path.join('./ckpt/', 'model.ckpt')
                self.saver.save(sess, checkpoint_path, global_step=int(e+1))
        return total_loss,total_correct 
        
    def train(self):
        print('Training')
        self.run_model(self.sess,self.y_out,self.mean_loss,self.X_train,self.y_train,\
            epochs=10,batch_size=64,print_every=50,save_every=1,\
            train_op=self.train_op,plot_losses=False)
        print('Validation')
        self.run_model(self.sess,self.y_out,self.mean_loss,self.X_val,self.y_val,\
            epochs=1,batch_size=64)
        print('Testing')
        self.run_model(self.sess,self.y_out,self.mean_loss,self.X_test,self.y_test,\
            epochs=1,batch_size=64)
    


if __name__=="__main__":
    m = Model()
    m.load_cifar10()
    m.construct_model(ckpt_dir="ckpt")
    m.train()

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,)
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)
convd
convd_mid
Tensor("Placeholder:0", shape=(?, 32, 32, 3), dtype=float32)
32 3 3 2 2
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

convd_end
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Please use `layer.__call__` method instead.
hello1
conv_1x1
convd
convd_mid


# New Section

In [0]:
%ls -alrt 
#%cp -rf ckpt ../../drive/*/.
%pwd



total 44
drwxr-xr-x 2 2156 1103 4096 Jun  4  2009 cifar-10-batches-py/
drwx------ 3 root root 4096 Mar 11 18:45 cs231n/
-rw------- 1 root root 1131 Mar 11 18:45 data.py
-rw------- 1 root root  134 Mar 11 18:45 get_datasets.sh
-rw------- 1 root root  642 Mar 11 18:46 model.py
-rw------- 1 root root 1618 Mar 11 18:46 prune_utils.py
-rw------- 1 root root 8187 Mar 11 18:46 train.py
drwx------ 2 root root 4096 Mar 11 18:46 ckpt/
drwx------ 5 root root 4096 Mar 11 18:46 ../
drwx------ 5 root root 4096 Mar 11 18:50 ./


NameError: ignored

# New Section

# New Section